# Deploy and Run the Hamiltonian Simulation Function

This interactive guide shows how to upload the hamiltonian simulation function to Qiskit Serverless and run an example workload.

### Requirements

This guide was developed with the following local package versions:

```
qiskit == 1.4.3
qiskit-ibm-catalog == 0.6.0
qiskit-ibm-runtime == 0.39.0
numpy == 2.2.6
```

## 1. Autentication

Use `qiskit-ibm-catalog` to authenticate to `QiskitServerless` with your API key (token), which you can find on the [IBM Quantum Platform](https://quantum.cloud.ibm.com) dashboard. This will allow you to locally instantiate the serverless client to upload or run the selected function:

```python
from qiskit_ibm_catalog import QiskitServerless
serverless = QiskitServerless(token="MY_TOKEN")
```

You can optionally use `save_account()` to save your credentials in your local environment (see the [Set up your IBM Cloud account](/docs/guides/cloud-setup#cloud-save) guide). Note that this writes your credentials to the same file as [`QiskitRuntimeService.save_account()`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#save_account):

```python
QiskitServerless.save_account(token="MY_TOKEN")
```

If the account is saved, there is no need to provide the token to authenticate:

In [1]:
from qiskit_ibm_catalog import QiskitServerless

# Authenticate to the remote cluster
# In this case, using the "ibm_quantum" channel
serverless = QiskitServerless(
    channel="ibm_quantum",
    token="49a6dd5171f88b09ee02affa7b06e7abfe346d6562f52d8151f972ac592ff69eaae03d2b954d6f686c6c27046ad7c3f993b74777cca360c5a4f5191984a0f0db",
)

## 2. Upload the custom function

To upload a Qiskit Function, you must first instantiate a `QiskitFunction` object that defines the function source code. The title will allow you to identify the function once it's in the remote cluster. The main entry point is the file that contains `if __name__ == "__main__"`. If your workflow requires additional source files, you can define a working directory that will be uploaded together with the entry point.

In [2]:
from qiskit_ibm_catalog import QiskitFunction

template = QiskitFunction(
    title="hamiltonian_simulation_template",
    entrypoint="hamiltonian_sim_entrypoint.py",
    working_dir="./source_files/",  # all files in this directory will be uploaded
)
print(template)

QiskitFunction(hamiltonian_simulation_template)


Once the instance is ready, upload it to serverless:

In [3]:
serverless.upload(template)

QiskitFunction(hamiltonian_simulation_template)

To check if the program successfully uploaded, use `serverless.list()`:

In [4]:
serverless.list()

[QiskitFunction(template_hamiltonian_simulation),
 QiskitFunction(hamiltonian_simulation_template),
 QiskitFunction(sqd_pcm_template)]

## 3. Lod and run the custom function remotely


The function template has been uploaded, so you can run it remotely with Qiskit Serverless. First, load the template by name:

In [5]:
template = serverless.load("hamiltonian_simulation_template")
print(template)

QiskitFunction(hamiltonian_simulation_template)


Next, run the template with the domain-level inputs for Hamiltonian simulation. This example specifies a 50-qubit XXZ model with random couplings, and an initial state and observable.

In [6]:
from itertools import chain
import numpy as np
from qiskit.quantum_info import SparsePauliOp

L = 50

# Generate the edge list for this spin-chain
edges = [(i, i + 1) for i in range(L - 1)]
# Generate an edge-coloring so we can make hw-efficient circuits
edges = edges[::2] + edges[1::2]

# Generate random coefficients for our XXZ Hamiltonian
np.random.seed(0)
Js = np.random.rand(L - 1) + 0.5 * np.ones(L - 1)

hamiltonian = SparsePauliOp.from_sparse_list(
    chain.from_iterable(
        [
            [
                ("XX", (i, j), Js[i] / 2),
                ("YY", (i, j), Js[i] / 2),
                ("ZZ", (i, j), Js[i]),
            ]
            for i, j in edges
        ]
    ),
    num_qubits=L,
)
observable = SparsePauliOp.from_sparse_list([("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L)

In [7]:
from qiskit import QuantumCircuit

initial_state = QuantumCircuit(L)
for i in range(L):
    if i % 2:
        initial_state.x(i)

In [8]:
# set up AQC options

aqc_options = {
    "aqc_evolution_time": 0.2,
    "aqc_ansatz_num_trotter_steps": 1,
    "aqc_target_num_trotter_steps": 32,
    "remainder_evolution_time": 0.2,
    "remainder_num_trotter_steps": 4,
    "aqc_max_iterations": 300,
}

job = template.run(
    backend_name="ibm_fez",
    hamiltonian=hamiltonian,
    observable=observable,
    initial_state=initial_state,
    aqc_options=aqc_options,
    estimator_options={},
    dry_run=True,
)
print(job.job_id)

907d33a2-3c43-49e7-a91e-5b5c26c0b7ae


Check the detailed status of the job:

In [9]:
import time

t0 = time.time()
status = job.status()
if status == "QUEUED":
    print(f"time = {time.time()-t0:.2f}, status = QUEUED")
while True:
    status = job.status()
    if status == "QUEUED":
        continue
    print(f"time = {time.time()-t0:.2f}, status = {status}")
    if status == "DONE" or status == "ERROR":
        break

time = 2.71, status = QUEUED
time = 73.89, status = INITIALIZING
time = 77.08, status = RUNNING
time = 79.62, status = RUNNING
time = 82.80, status = RUNNING
time = 85.66, status = RUNNING
time = 88.30, status = RUNNING
time = 91.12, status = RUNNING
time = 93.83, status = RUNNING
time = 96.42, status = RUNNING
time = 98.98, status = RUNNING
time = 101.74, status = RUNNING
time = 104.40, status = RUNNING
time = 107.11, status = RUNNING
time = 109.73, status = RUNNING
time = 112.39, status = RUNNING
time = 114.99, status = RUNNING
time = 117.72, status = RUNNING
time = 120.38, status = RUNNING
time = 123.25, status = RUNNING
time = 125.91, status = RUNNING
time = 128.67, status = RUNNING
time = 131.47, status = RUNNING
time = 134.03, status = RUNNING
time = 136.67, status = RUNNING
time = 139.62, status = RUNNING
time = 142.40, status = RUNNING: MAPPING
time = 145.16, status = RUNNING: MAPPING
time = 147.92, status = RUNNING: MAPPING
time = 150.89, status = RUNNING: MAPPING
time = 153.4

After the job is running, you can fetch logs created from the `logger.info` outputs. These can provide actionable information about the progress of the Hamiltonian simulation workflow. For example, the value of the objective function during the iterative component of AQC, or the two-qubit depth of the final ISA circuit intended for execution on hardware.

In [10]:
print(job.logs())

2025-05-26 13:28:05,191	INFO job_manager.py:531 -- Runtime env is setting up.
/usr/local/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:54: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:73: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/usr/local/lib64/python3.11/site-packages/qiskit/qpy/interface.py:346: UserWarning: The qiskit version used to generate the provided QPY file, 1.4.3, is newer than the current qiskit version 1.4.2. This may result in an error if the QPY file uses instructions not present in this current qiskit version
  warnings.warn(
hamiltonian_sim_entrypoint.run_function:INFO:2025-05-26 13:29:05,025: Starting runtime service
/tmp/ray/session_2025-05-26_13-27-38_752650_1/runtime

Block the rest of the program until a result is available. After the job is done, you can retrieve the results. These include the domain-level output of Hamiltonian simulation (expectation value) and useful metadata.

In [11]:
result = job.result()

del result["aqc_final_parameters"]  # the list is too long to conveniently display here
result

{'target_bond_dimension': 5,
 'num_aqc_parameters': 816,
 'aqc_starting_fidelity': 0.9914382555614002,
 'num_iterations': 55,
 'aqc_fidelity': 0.9997823400276018,
 'twoqubit_depth': 33,
 'metadata': {'resources_usage': {'RUNNING: MAPPING': {'CPU_TIME': 52.6219527721405},
   'RUNNING: OPTIMIZING_FOR_HARDWARE': {'CPU_TIME': 10.403448820114136}}}}

Note that the result metadata includes a resource usage summary that allows to better estimate the QPU and CPU time required for each workload (this example runs on dry-run mode so no quantum resources are used).  

After the job completes, the entire logging output will be available.

In [12]:
print(job.logs())

2025-05-26 13:28:05,191	INFO job_manager.py:531 -- Runtime env is setting up.
/usr/local/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:54: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:73: UserWarning: Couldn't find `optuna`, `cmaes`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/usr/local/lib64/python3.11/site-packages/qiskit/qpy/interface.py:346: UserWarning: The qiskit version used to generate the provided QPY file, 1.4.3, is newer than the current qiskit version 1.4.2. This may result in an error if the QPY file uses instructions not present in this current qiskit version
  warnings.warn(
hamiltonian_sim_entrypoint.run_function:INFO:2025-05-26 13:29:05,025: Starting runtime service
/tmp/ray/session_2025-05-26_13-27-38_752650_1/runtime